In [157]:
import sqlite3
from datetime import datetime, timedelta
import random
import logging
import time
import pandas as pd


In [158]:
logging.basicConfig(
    level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s'
    )


In [159]:
def generar_datos(num_transacciones):
    inicio_fecha = datetime(2022, 1, 1)
    fin_fecha = datetime(2023, 1, 1)
    datos = []
    id_transaccion = 1
    contador_transacciones = 0

    while contador_transacciones < num_transacciones:
        fecha = inicio_fecha + timedelta(
            days=random.randint(0, (fin_fecha - inicio_fecha).days)
            )
        credito = round(random.uniform(100, 10000), 2)
        num_debitos = random.randint(1, 5)
        valor_base_debito = credito / num_debitos
        suma_debitos = 0

        for _ in range(num_debitos - 1):
            variacion = random.uniform(-10, 10)
            debito = round(valor_base_debito + variacion, 2)
            suma_debitos += debito
            datos.append(
                (id_transaccion, fecha.strftime("%Y-%m-%d"), debito, 0)
                )
            id_transaccion += 1

        ultimo_debito = round(credito - suma_debitos, 2)
        datos.append(
            (id_transaccion, fecha.strftime("%Y-%m-%d"), ultimo_debito, 0)
            )
        id_transaccion += 1

        datos.append((id_transaccion, fecha.strftime("%Y-%m-%d"), 0, credito))
        id_transaccion += 1
        contador_transacciones += 1

    fecha = fin_fecha - timedelta(days=1)
    debito = sum(d[2] for d in datos)
    credito = sum(d[3] for d in datos)
    datos.append(
        (id_transaccion,
         fecha.strftime("%Y-%m-%d"),
         (debito - credito) if debito > credito else 0,
         (credito - debito) if credito > debito else 0)
        )
    return datos

In [160]:
def tabla_de_prueba():
    # Conectar a la base de datos SQLite
    conn = sqlite3.connect('transacciones.db')
    c = conn.cursor()

    # Crear tabla
    c.execute(
        '''CREATE TABLE IF NOT EXISTS transacciones
                         (id_transaccion INTEGER PRIMARY KEY, fecha TEXT, debito REAL, credito REAL)'''
        )

    # Borrar datos existentes
    c.execute('DELETE FROM transacciones')

    # Generar datos
    datos = generar_datos(10)

    # Insertar datos en la tabla
    c.executemany('INSERT INTO transacciones VALUES (?,?,?,?)', datos)

    # Guardar (commit) los cambios y cerrar la conexión
    conn.commit()
    conn.close()

In [161]:
def obtener_creditos_y_debitos():
    conn = sqlite3.connect('transacciones.db')
    cursor = conn.cursor()
    cursor.execute(
        "SELECT id_transaccion, debito FROM transacciones WHERE debito > 0"
        )
    debitos = cursor.fetchall()
    cursor.execute(
        "SELECT id_transaccion, credito FROM transacciones WHERE credito > 0"
        )
    creditos = cursor.fetchall()
    conn.close()
    return creditos, debitos


In [162]:
df_transacciones = pd.read_excel('transacciones.xlsx')
df_transacciones

,Registros,TRAACR,TRACCY,TRACDE,Db,Cr,TotHNL
0,3,66736761,LPS,T8,37505.13,37598.89,-93.76
1,3,66738801,LPS,T8,1251.54,1261.54,-10.00
2,2,66738871,LPS,T8,0.00,574.15,-574.15
3,1,66738874,LPS,T8,564.15,0.00,564.15
4,2,66738941,LPS,T8,0.00,1170.79,-1170.79
...,...,...,...,...,...,...,...
89,3,66773611,LPS,T8,46086.61,46201.82,-115.21
90,2,66774261,LPS,T8,0.00,10492.00,-10492.00
91,1,66774265,LPS,T8,10465.84,0.00,10465.84
92,2,66774501,LPS,T8,0.00,133.74,-133.74


In [163]:
df_transacciones.groupby('TRAACR').count().sort_values(
    'TRAACR', ascending=False
    )

,Registros,TRACCY,TRACDE,Db,Cr,TotHNL
TRAACR,,,,,,
66774503,1,1,1,1,1,1
66774501,1,1,1,1,1,1
66774265,1,1,1,1,1,1
66774261,1,1,1,1,1,1
66773611,1,1,1,1,1,1
...,...,...,...,...,...,...
66738941,1,1,1,1,1,1
66738874,1,1,1,1,1,1
66738871,1,1,1,1,1,1


In [164]:
# Cambiar el nombre de las columnas
df_transacciones = df_transacciones.rename(
    columns={'TRAACR': 'id_transaccion', 'Db': 'debito', 'Cr': 'credito'}
    )

df_transacciones = df_transacciones[['id_transaccion', 'debito', 'credito']]

# id_transaccion debe ser texto
df_transacciones['id_transaccion'] = df_transacciones['id_transaccion'].astype(str)

In [165]:
df_transacciones

,id_transaccion,debito,credito
0,66736761,37505.13,37598.89
1,66738801,1251.54,1261.54
2,66738871,0.00,574.15
3,66738874,564.15,0.00
4,66738941,0.00,1170.79
...,...,...,...
89,66773611,46086.61,46201.82
90,66774261,0.00,10492.00
91,66774265,10465.84,0.00
92,66774501,0.00,133.74


In [166]:
def obtener_datos_de_excel():
    df = pd.read_excel('transacciones.xlsx')
    df = df.rename(
        columns={'TRAACR': 'id_transaccion', 'Db': 'debito', 'Cr': 'credito'}
        )
    df = df[['id_transaccion', 'debito', 'credito']]
    df['id_transaccion'] = df['id_transaccion'].astype(str)
    creditos = df[df['credito'] > 0][
        ['id_transaccion', 'credito']].values.tolist()
    debitos = df[df['debito'] > 0][['id_transaccion', 'debito']].values.tolist()
    return creditos, debitos


df_creditos, df_debitos = obtener_datos_de_excel()

In [167]:
def encontrar_combinacion_optima(
        debitos, credito, start=0, current_sum=0, current_comb=[],
        tolerancia=1.0
        ):
    logging.debug(
        f"Iniciando encontrar_combinacion_optima con credito={credito}, start={start}, current_sum={current_sum}, tolerancia={tolerancia}"
        )
    # if current_sum == credito:
    if abs(current_sum - credito) <= tolerancia:
        logging.debug(f"Combinación encontrada: {current_comb}")
        return current_comb
    if current_sum > credito or start >= len(debitos):
        return None
    incluir = encontrar_combinacion_optima(
        debitos, credito, start + 1, current_sum + debitos[start][1],
                          current_comb + [debitos[start]]
        )
    if incluir is not None:
        return incluir
    excluir = encontrar_combinacion_optima(
        debitos, credito, start + 1, current_sum, current_comb
        )
    return excluir


In [168]:
def backtracking_combinacion_optima(
        debitos, credito, start=0, current_sum=0, current_comb=[],
        tolerancia=0.05
        ):
    if abs(current_sum - credito) <= tolerancia:
        return current_comb
    if current_sum > credito or start >= len(debitos):
        return None
    for i in range(start, len(debitos)):
        incluir = backtracking_combinacion_optima(
            debitos, credito, i + 1, current_sum + debitos[i][1],
                              current_comb + [debitos[i]]
            )
        if incluir is not None:
            return incluir
    return None

In [169]:
def emparejar_creditos_debitos():
    logging.info("Iniciando emparejar_creditos_debitos")
    creditos, debitos = obtener_datos_de_excel()
    emparejamientos = []
    for credito in creditos:
        logging.debug(f"Procesando credito: {credito}")
        combinacion = encontrar_combinacion_optima(debitos, credito[1])
        if combinacion:
            emparejamientos.append((credito, combinacion))
            for debito in combinacion:
                debitos.remove(debito)
    logging.info(
        f"Emparejamientos completados: {len(emparejamientos)} encontrados"
        )
    return emparejamientos


In [170]:
def emparejar_creditos_debitos_backtracking():
    logging.info("Iniciando emparejar_creditos_debitos con backtracking")
    creditos, debitos = obtener_datos_de_excel()
    emparejamientos = []
    for credito in creditos:
        combinacion = backtracking_combinacion_optima(debitos, credito[1])
        if combinacion:
            emparejamientos.append((credito, combinacion))
            for debito in combinacion:
                debitos.remove(debito)
    logging.info(
        f"Emparejamientos completados: {len(emparejamientos)} encontrados"
        )
    return emparejamientos

In [174]:
def buscar_debitos_segun_cantidad(cantidad):
    _, debitos = obtener_datos_de_excel()
    combinacion = encontrar_combinacion_optima(debitos, cantidad)
    return combinacion

In [175]:
def main():
    print("Emparejamiento de créditos y débitos")
    start_time = time.time()
    emparejamientos = emparejar_creditos_debitos()
    end_time = time.time()
    print(f"Tiempo de ejecución: {end_time - start_time} segundos")
    for emparejamiento in emparejamientos:
        print(emparejamiento)
    # agrupar las transacciones en un solo dataframe para guardarlas en un archivo Excel
    rows = []
    for group_id, (credito, debitos) in enumerate(emparejamientos, start=1):
        balance = 0
        # Agregar el crédito con su grupo
        balance += credito[1]
        rows.append((group_id, credito[0], 0, credito[1], balance))
        # Agregar cada débito con el mismo grupo
        for debito in debitos:
            balance -= debito[1]
            rows.append((group_id, debito[0], debito[1], 0, balance))
        rows.append((group_id, 'balance', 0, 0, balance))
    df_emparejamientos = pd.DataFrame(rows, columns=['group_id', 'id_transaccion', 'debito', 'credito', 'balance'])
    # guardar los resultados en un archivo Excel
    df_emparejamientos.to_excel('emparejamientos.xlsx', index=False)

    emparejamiento_backtracking = emparejar_creditos_debitos_backtracking()
    for emparejamiento in emparejamiento_backtracking:
        print(emparejamiento)
    rows = []
    for group_id, (credito, debitos) in enumerate(emparejamiento_backtracking, start=1):
        balance = 0
        # Agregar el crédito con su grupo
        balance += credito[1]
        rows.append((group_id, credito[0], 0, credito[1], balance))
        # Agregar cada débito con el mismo grupo
        for debito in debitos:
            balance -= debito[1]
            rows.append((group_id, debito[0], debito[1], 0, balance))
        rows.append((group_id, 'balance', 0, 0, balance))
    df_emparejamientos = pd.DataFrame(rows, columns=['group_id', 'id_transaccion', 'debito', 'credito', 'balance'])
    # guardar los resultados en un archivo Excel
    df_emparejamientos.to_excel('emparejamientos_backtracking.xlsx', index=False)
    
    emparejamiento_cantidad = buscar_debitos_segun_cantidad(cantidad= 263_257.82)
    print(emparejamiento_cantidad)
    rows = []
    group_id = 1
    balance = 0
    for debito in emparejamiento_cantidad:
        balance -= debito[1]
        rows.append((group_id, debito[0], debito[1], 0, balance))
    rows.append((group_id, 'balance', 0, 0, balance))
    df_emparejamientos = pd.DataFrame(rows, columns=['group_id', 'id_transaccion', 'debito', 'credito', 'balance'])
    # guardar los resultados en un archivo Excel
    df_emparejamientos.to_excel('emparejamientos_cantidad.xlsx', index=False)
    
    



In [176]:
if __name__ == '__main__':
    # establecer el nivel de registro a INFO o DEBUG para ver los mensajes de depuración
    logging.getLogger().setLevel(logging.INFO)
    # tabla_de_prueba()
    main()

[['66736761', 37505.13], ['66738801', 1251.54], ['66738874', 564.15], ['66738949', 1160.79], ['66739032', 481.61], ['66739120', 1470.52], ['66739209', 1690.07], ['66740236', 123.74], ['66740513', 123.74], ['66746774', 113.85], ['66746958', 12247.3], ['66749335', 123.74], ['66749648', 1237.43], ['66751670', 123.74], ['66751908', 53100.44], ['66752955', 68297.01], ['66754707', 371.23], ['66756103', 1682.14], ['66760598', 1958.74], ['66761888', 8081.04], ['66762043', 3729.68], ['66769152', 8296.9], ['66772990', 2969.84], ['66773611', 46086.61], ['66774265', 10465.84]]
